In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv("IMDB Dataset.csv", on_bad_lines='skip')

In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
data["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [ ]:

# LSTM -> LONG SHORT TERM MEMORY
# RNN -> TEXTUAL DATA

In [ ]:

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
train_data, test_data = train_test_split(data, test_size = 0.2, random_state=42)

In [ ]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data["review"])

In [ ]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [ ]:
Y_train = train_data["sentiment"].map({"positive": 1, "negative": 0})
Y_test = test_data["sentiment"].map({"positive": 1, "negative": 0})

In [ ]:
# LSTM MODEL BUILDING

In [ ]:
model = Sequential()
model.add(Embedding(input_dim =5000, output_dim = 128, input_length = 200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout = 0.2))
model.add(Dense(1, activation = "sigmoid"))

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (64, 200, 128)              │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (64, 128)                   │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (64, 1)                     │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,315,141 (8.83 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,543,428 (5.89 MB)

In [ ]:
model.compile(optimizer = "adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
model.fit(X_train, Y_train, epochs = 5, batch_size = 64, validation_split = 0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 121s 239ms/step - accuracy: 0.7291 - loss: 0.5223 - val_accuracy: 0.8428 - val_loss: 0.3674
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 142s 238ms/step - accuracy: 0.8218 - loss: 0.4006 - val_accuracy: 0.8200 - val_loss: 0.4139
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 142s 240ms/step - accuracy: 0.8648 - loss: 0.3214 - val_accuracy: 0.8830 - val_loss: 0.2792
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 119s 239ms/step - accuracy: 0.9107 - loss: 0.2243 - val_accuracy: 0.8846 - val_loss: 0.2763
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 119s 239ms/step - accuracy: 0.9307 - loss: 0.1846 - val_accuracy: 0.8860 - val_loss: 0.2830


In [ ]:
model.save("model.h5")

In [ ]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

['tokenizer.pkl']

In [ ]:
loss, accuracy = model.evaluate(X_test, Y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 26s 83ms/step - accuracy: 0.8885 - loss: 0.2724


In [ ]:
print(loss)

0.26893046498298645


In [1]:
def predictive_system(review):
  sequences = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequences, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "neutral" if prediction[0][0] == 0.5 else "negative"
  return sentiment

In [ ]:
predictive_system("This movie was fantastic and amazing")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step


'positive'

In [ ]:
predictive_system("A trilling adventure with stunning visual")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step


'positive'

In [ ]:
predictive_system("mid")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step


'negative'

In [ ]:
predictive_system("not too bad nor too good")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step


'negative'

In [ ]:
predictive_system("Not too bad to watch")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step


'negative'